# STAGE 2 - Automated Data Ingestion and Cleaning: Excel to MySQL

**Import Libraries:**
* **pandas:** Used for reading and processing Excel files.
* **pymysql:** Connects and interacts with a MySQL database.

In [1]:
import pandas as pd
import pymysql

**Define Company Details:**

Each company has a file path (to its Excel file), a company name, and a table name.

In [2]:
# List of companies with their respective Excel file paths and table names
companies = [{"file_path": r"E:\Stock Data\Indian Hotels.xlsx", "company_name": "Indian Hotels", "table_name": "Indian_Hotels"},
    {"file_path": r"E:\Stock Data\Nelco.xlsx", "company_name": "Nelco", "table_name": "Nelco"},
    {"file_path": r"E:\Stock Data\Rallis India.xlsx", "company_name": "Rallis India", "table_name": "Rallis_India"},
    {"file_path": r"E:\Stock Data\Tata Chemicals.xlsx", "company_name": "Tata Chemicals", "table_name": "Tata_Chemicals"},
    {"file_path": r"E:\Stock Data\Tata Power.xlsx", "company_name": "Tata Power", "table_name": "Tata_Power"},
    {"file_path": r"E:\Stock Data\Tata Communication.xlsx", "company_name": "Tata Communication", "table_name": "Tata_Communication"},
    {"file_path": r"E:\Stock Data\Tata Consumer.xlsx", "company_name": "Tata Consumer", "table_name": "Tata_Consumer"},
    {"file_path": r"E:\Stock Data\Tata Elxsi.xlsx", "company_name": "Tata Elxsi", "table_name": "Tata_Elxsi"},
    {"file_path": r"E:\Stock Data\Tata Investment Corp.xlsx", "company_name": "Tata Investment Corp", "table_name": "Tata_Investment_Corp"},
    {"file_path": r"E:\Stock Data\Tata Motors.xlsx", "company_name": "Tata Motors", "table_name": "Tata_Motors"},
    {"file_path": r"E:\Stock Data\Titan.xlsx", "company_name": "Titan", "table_name": "Titan"},
    {"file_path": r"E:\Stock Data\Tata Steel.xlsx", "company_name": "Tata Steel", "table_name": "Tata_Steel"},
    {"file_path": r"E:\Stock Data\Trent.xlsx", "company_name": "Trent", "table_name": "Trent"},
    {"file_path": r"E:\Stock Data\Voltas.xlsx", "company_name": "Voltas", "table_name": "Voltas"}]

**Database Configuration:**

Stores the database connection details.

In [3]:
# Database configuration
db_config = {'host': 'localhost', 'user': 'root', 'password': 'Onmyway09@', 'database': 'STOCK_PREDICTION'}

**The `process_company_data` function processes stock data for a single company. Here's what it does:**

1. **Reads the Excel File**: Loads the company's stock data from the specified fil.

2. **Cleans Column Names**: Strips extra spaces from column headrs.

3. **Formats Dates**: Converts the 'Date' column to a standard YYYY-MM-DD format and removes invalid enries.

4. **Cleans Data**: Removes commas from numeric columns and converts them to proper numeri types.

5. **Handles Missing Values**: Replaces NaN valus with 0.

6. **Adds Company Name**: Adds a column to label the data with the comany's name.

7. **Connects to MySQL**: Establishes a databae connection.

8. **Creates a Table**: Ensures a table for the company's data exists n the database.

9. **Inserts Data**: Adds the cleaned data to te database table.

10. **Handles Errors**: Logs any database errors and ensures conheir data efficiently.

In [4]:
# Function to process and insert data for a single company
def process_company_data(company):
    file_path = company["file_path"]
    company_name = company["company_name"]
    table_name = company["table_name"]

    # Read the Excel file
    try:
        data = pd.read_excel(file_path)
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return

    # Clean column names by stripping leading/trailing spaces
    data.columns = data.columns.str.strip()

    # Clean the data
    # Convert 'Date' to datetime format and ensure only YYYY-MM-DD format
    data['Date'] = pd.to_datetime(data['Date'], errors='coerce').dt.strftime('%Y-%m-%d')
    data = data[data['Date'].notna()]  # Drop rows with invalid dates

    # Clean columns with commas or non-numeric characters
    columns_to_clean = ['52W H', 'VOLUME', 'VALUE']
    for col in columns_to_clean:
        data[col] = data[col].astype(str).str.replace(',', '', regex=False)
        data[col] = pd.to_numeric(data[col], errors='coerce')

    # Ensure numeric fields are converted to float where applicable
    numeric_columns = ['OPEN', 'HIGH', 'LOW', 'PREV. CLOSE', 'ltp', 'close', 'vwap', '52W L']
    for col in numeric_columns:
        data[col] = pd.to_numeric(data[col], errors='coerce')

    # Add company name to the data
    data['Company'] = company_name

    # Handle NaN values
    data.fillna(0, inplace=True)

    # Connect to the MySQL database and insert data
    connection = None
    try:
        connection = pymysql.connect(**db_config)
        cursor = connection.cursor()

        # Create a table (if it doesn't exist)
        create_table_query = f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            Date DATE NOT NULL,
            series TEXT NOT NULL,
            OPEN DOUBLE, HIGH DOUBLE, LOW DOUBLE,
            `PREV. CLOSE` DOUBLE, ltp DOUBLE, close DOUBLE, vwap DOUBLE,
            `52W H` TEXT, `52W L` DOUBLE, VOLUME TEXT, VALUE TEXT,
            `No of trades` TEXT, Company TEXT
        );"""
        cursor.execute(create_table_query)
        print(f"Table '{table_name}' is ready.")

        # Insert data into the MySQL table
        insert_query = f"""
        INSERT INTO {table_name} (Date, series, OPEN, HIGH, LOW, `PREV. CLOSE`, ltp, close, vwap,
                                  `52W H`, `52W L`, VOLUME, VALUE, `No of trades`, Company)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        for _, row in data.iterrows():
            cursor.execute(insert_query, (row['Date'], row['series'], row['OPEN'],
                row['HIGH'], row['LOW'], row['PREV. CLOSE'],
                row['ltp'], row['close'], row['vwap'],
                row['52W H'], row['52W L'], row['VOLUME'],
                row['VALUE'], row['No of trades'], row['Company']))

        # Commit the transaction
        connection.commit()
        print(f"Data inserted successfully into the '{table_name}' table for {company_name}.")
        print("\n")

    except pymysql.MySQLError as err:
        print(f"Error for {company_name}: {err}")
    finally:
        if connection:
            connection.close()

# Iterate over all companies and process their data
for company in companies:
    process_company_data(company)

Table 'Indian_Hotels' is ready.
Data inserted successfully into the 'Indian_Hotels' table for Indian Hotels.


Table 'Nelco' is ready.
Data inserted successfully into the 'Nelco' table for Nelco.


Table 'Rallis_India' is ready.
Data inserted successfully into the 'Rallis_India' table for Rallis India.


Table 'Tata_Chemicals' is ready.
Data inserted successfully into the 'Tata_Chemicals' table for Tata Chemicals.


Table 'Tata_Power' is ready.
Data inserted successfully into the 'Tata_Power' table for Tata Power.


Table 'Tata_Communication' is ready.
Data inserted successfully into the 'Tata_Communication' table for Tata Communication.


Table 'Tata_Consumer' is ready.
Data inserted successfully into the 'Tata_Consumer' table for Tata Consumer.


Table 'Tata_Elxsi' is ready.
Data inserted successfully into the 'Tata_Elxsi' table for Tata Elxsi.


Table 'Tata_Investment_Corp' is ready.
Data inserted successfully into the 'Tata_Investment_Corp' table for Tata Investment Corp.


Table '